In [91]:
#!/usr/bin/env python
# coding: utf-8
import nltk
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx
import os
import xlwt
import xlrd
from xlutils.copy import copy


def read_article(file_name):
    file = open(file_name, "r")
    filedata = file.readlines()
    article = []
    title = ''
    for eachLine in filedata:
        eachLine = eachLine[:-2]
        if len(eachLine) >= 1:
            if eachLine[:3] != 'URL' and eachLine[:2] != 'ID' and eachLine[:4] != 'Date' and eachLine[:8] != 'Ariticle':
                if eachLine[:5] == 'Title':
                    title += eachLine[8:]
#                     article += title.split(". ")
                else:
#                     print('test: ',eachLine)
                    article += eachLine.split(". ")
    sentences = []

    for sentence in article:
#         print('sentence: ', sentence)
        test = sentence.replace("[^a-zA-Z]", " ")
#         print('test: ', test)
        afterToken = test.split(" ")  
#         print('afterToken: ', sentence)
        if len(afterToken) > 8:
            sentences.append(afterToken)
    if len(sentences) > 1:
        sentences.pop()
    return sentences,title

def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []
 
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
 
    all_words = list(set(sent1 + sent2))
 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
 
    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
 
    return 1 - cosine_distance(vector1, vector2)
 
def build_similarity_matrix(sentences, stop_words):
    # Create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
 
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2: #ignore if both are same sentences
                continue 
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)

    return similarity_matrix


def generate_summary(file_name, top_n=5):
    nltk.download("stopwords")
    stop_words = stopwords.words('english')
    summarize_text = []

    # Step 1 - Read text anc split it
    sentences,title =  read_article(file_name)

    # Step 2 - Generate Similary Martix across sentences
    sentence_similarity_martix = build_similarity_matrix(sentences, stop_words)

    # Step 3 - Rank sentences in similarity martix
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
    scores = nx.pagerank(sentence_similarity_graph)

    # Step 4 - Sort the rank and pick top sentences
    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)    
#     print("Indexes of top ranked_sentence order are ", ranked_sentence)    
    
    if len(ranked_sentence) < top_n:
        for k in range(len(ranked_sentence)):
            if ranked_sentence[k][1].count('|') < 2:
                summarize_text.append(" ".join(ranked_sentence[k][1]))
    else:
        i = 0
        count = 0
        while count < top_n and i < len(ranked_sentence) :
            if ranked_sentence[i][1].count('|') < 2:
                summarize_text.append(" ".join(ranked_sentence[i][1]))
#                 print(count,': ',summarize_text)
                count += 1
            i += 1
            
    # Step 5 - Offcourse, output the summarize texr
    final_summarize_text = ". ".join(summarize_text)
    if len(final_summarize_text) == 0:
        final_summarize_text += title
#     print("Summarize Text: \n", final_summarize_text)
    return final_summarize_text

# let's begin
# directory = "../"
# os.chdir(directory)  #切换到directory目录
# cwd = os.getcwd()  #获取当前目录即dir目录下
# print(cwd)
# directory = "./FinalProject"
# os.chdir(directory)  #切换到directory目录
# cwd = os.getcwd()  #获取当前目录即dir目录下
# print(cwd)
booklist = xlrd.open_workbook('PreprocessedNewsList.xls')
sheet = booklist.sheet_by_index(0)
fileNames = (sheet.col_values(0))
directory = "./NewsContents"
os.chdir(directory)  #切换到directory目录
cwd = os.getcwd()  #获取当前目录即dir目录下
print(cwd)
files = os.listdir(cwd)  #列出目录下的文件


newWb = copy(booklist)
newWs = newWb.get_sheet(0);#取sheet表
count = 1
for eachFile in files:
    print(count,': ',eachFile)
    sumText = generate_summary( eachFile, 3)
    print(count,': ',sumText)
#     if count > 2:
#         break
    newWs.write(fileNames.index(eachFile[:-4]),2,sumText)
    count += 1

directory = "../"
os.chdir(directory)  #切换到directory目录
cwd = os.getcwd()  #获取当前目录即dir目录下
print(cwd)
newWb.save('NLTKSumNews.xls')


# sumText = generate_summary( 'HMAP2020042900532.txt', 3)
# print(sumText)

/Users/WenhuiJin/Desktop/2020 Spring/CSE519/FinalProject/NewsContents
1 :  GPHN2020050200011.txt


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/WenhuiJin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


1 :  In Quebec, 373 cases came from the United States and the province reports zero cases connected to travel from China. She said curbs on travel from Iran came before Iran was even reporting it had cases because the public health data in Canada made it clear there was a problem there. Air Canada suspended flights from China in February and the government encouraged people not to travel to China as early as January, but did not ban travellers until March 18 when it imposed sweeping global restrictions
2 :  HMAP2020042900276.txt
2 :  SF General Boasts Only One Death to Date After Treating 38 COVID-19 Patients In Intensive Care ; ;
3 :  ALJA2020042900007.txt
3 :  Hoyer said lawmakers will be asked to return when the legislation is complete. Capitol physician warns lawmakers that it will be at least a year before Congress can return to business as usual. Hoyer said that because the Washington, DC area remains a "hot spot" under stay-at-home orders - with the numbers of cases "going up an

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/WenhuiJin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/WenhuiJin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/WenhuiJin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


4 :  In Italy, which has seen the world’s most deaths with more than 28,000, people looked enviously on at Spain as they awaited their own relaxation of restrictions now that the number of new cases has leveled out. Russia sees record spike in coronavirus cases with nearly 10,000 in one da. Russia sees record spike in coronavirus cases with nearly 10,000 in one da
5 :  ALFR2020050100092.txt
5 :  Drawn from the local curriculum, kids have an opportunity to learn via radio classrooms while parents continue home learning with provided materials following the show. Monrovia — It's another weekday when kids should be in school at Little Wlebo Public School in Maryland County, Southeast Liberia but all hopes are dashed due to the Coronavirus pandemic. Madam Lago Louise, a refugee now based in Liberia for almost a decade, hopes of having her kids learn in the free government school programme operated by Bridge Liberia seems all but ended
6 :  GPHN2020050300011.txt


...

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/WenhuiJin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/WenhuiJin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


6129 :  Presumptive Democratic presidential nominee Joe Biden won the endorsement on Tuesday of Hillary Clinton for the office she fell short of winning in 2016 in her bid to become the first woman elected as president of the United States. Clinton's endorsement comes at a critical moment as Biden aims to raise his profile with female voters. She also lost the 2008 Democratic presidential race to Barack Obama, whom Biden served as vice president
6130 :  GPHN2020050300020.txt
6130 :  Health officials reported that 66.5 per cent of cases in Ontario have now been resolved.. TORONTO  --   Ontario health officials are reporting 511 new cases of COVID-19 and 55 more deaths in the province. Of the more than 17,000 COVID-19 lab-confirmed cases reported in Ontario, health officials say about 11.6 per cent have been hospitalized at some point
6131 :  HMAP2020042900253.txt
6131 :  Oscars make one-time exception for streaming eligibility because of coronavirus ; ;
6132 :  GPHN2020050200034.txt
613

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/WenhuiJin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/WenhuiJin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/WenhuiJin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/WenhuiJin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/WenhuiJin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/WenhuiJin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


 :  The Heart and Stroke Foundation and Canadian Cardiovascular Society are worried that some people may be avoiding hospitals, even when faced with an emergency such as a serious heart attack, due to COVID-19. Fewer people in Ontario seeking help for serious heart attacks: Heart and Strok. Experts don’t think the number of serious heart attacks has actually fallen, however they are worried that heart patients are avoiding hospitals out of fear of catching COVID-19
6136 :  HMAP2020050100172.txt
6136 :  Coronavirus live updates: Pandemic weighs on tech earnings ; ;
6137 :  ALFR2020050200027.txt
6137 :  Nigeria has seen a spike in cases of coronavirus as it recorded over 500 cases in the last three days. Results of the tests by the agency showed that eight other members of the Dokpesi family, among whom are Mr Dokpesi's daughter-in-law, had been infected, AIT said. AIT also said the FCT authority has fumigated the company's premises
6138 :  GPHN2020050200008.txt
6138 :  In Nunavut, which

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/WenhuiJin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/WenhuiJin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/WenhuiJin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/WenhuiJin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
